In [ ]:
from methods import *

In [ ]:
# Make a single request
#getPostsToTimestamp("abu dhabi gp 2023", timestamp = 1701010800, replace = False, debug = True)

In [ ]:
%%time

# Get all posts for a race
getPostsForGP("2023-austrian-grand-prix", startAtTag = "austrian gp 2023", delay = 3600, replace = False)

In [ ]:
%%time
getPostsForSeason(debug = False, startAtRace = 13, raceLimit = 1, delay = 3600, replace = False)

## Post Length & Sentiment Analysis

In [ ]:
%%time

computePostLengthAndEmotionForRace("2023-bahrain-grand-prix", debug = True)

In [ ]:
%%time

computePostLengthAndEmotionForSeason(2023)

## Sentiment Analysis

In [ ]:
# Get emotion
emotion = get_emotion("congrats")
print("Emotion Score", emotion)


## Load Race JSON File

In [ ]:
loadSeasonFromWeb(debug = False)

## Analysis

In [ ]:
from dataAnalysis import *

In [ ]:
ttestLengthAndEmotion(0, 23, returning = "emotion", debug = True)

In [ ]:
ttestLengthAndEmotion(0, 100, returning = "length", debug = True)

In [ ]:
graphPValuePostCrash(timeAfterCrash = 60 * 1, returning = "emotion", caption = "Figure 2: P-Value of Absolute Emotion Score", step = 1)

In [ ]:
graphPValuePostCrash(timeAfterCrash = 60*6, returning = "length", caption = "Figure 3: P-Value of Post Length", step = 5)